숙명여대 학과수집

In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

In [2]:
# 수집할 컨텐츠가 있는 웹 페이지의 주소
url = "https://www.sookmyung.ac.kr/sookmyungkr/1030/subview.do"

In [3]:
# 접속객체 생성
session = requests.Session()
# 접속객체에 부가정보(header) 삽입하기
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
})
# 생성한 접속객체를 활용하여 API에 접속
r = session.get(url)
# 접속에 실패한 경우
if r.status_code != 200:
# 에러코드와 에러메시지 출력
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
# 에러를 강제로 생성시킴
    raise Exception(msg)
r.encoding = "utf-8"
#print(r.text)
soup = BeautifulSoup(r.text)
# soup

응답결과로부터 데이터 추출하기

In [5]:
# 수집한 정보를 저장할 빈 리스트
학과목록_리스트 = []
# 홈페이지에서 정보를 추출하는 경우 원하는 답을 얻을때까지 for문을
# 수정해가면서 하면 좋다.
# 그래서 중간중간 print로 값을 확인하며 오류가뜨면
# None값에 대한 처리? 범위를 벗어났는지? 등에 대해야 계속해서 수정을 해가야한다.
college_list = soup.select(".college_list")
for item in college_list:
    h5El = item.select("h5")
    name = h5El[0].text.strip()

    # 상세보기 URL
    # 가져온 페이지 주소의 예) /sookmyungkr/1012/subview.do
    # 같은 사이트 안에서 이동할 때는 앞부분 "https: / w.sookmyung.ac.kr"을 생략 가능하지만
    # 외부에서 직접 접근할 때는 반드시 전체 주소가 필요함.
    # 가져온 값이 전체주소가 아니므로 검사하여 누락된 부분을 덧붙여야 한다.
    viewEl = item.select(".view")
    view = viewEl[0].attrs["href"]
    if view.find("http://www.sookmyung.ac.kr") == -1:
        view = "http://www.sookmyung.ac.kr" + view


    # 학과소개 pdf
    pdfEl = item.select(".info") # soup.select하면 전체를 검토해서 info를가져오기 때문에
    pdf = pdfEl[0].attrs["href"] # item으로 한 학과 마다 검토를 해야 정확한 값이 나옴
    
    # 학과홈페이지
    homepageEl = item.select(".homepage")
    homepage = homepageEl[0].attrs['href']
    
    # 수집한 값들을 딕셔너리로 묶은 후 리스트에 추가
    college_dict = {"학과이름": name, "상세페이지": view, "홈페이지": homepage}

    #------------------------------------------------------# 상세보기 URL을 새롭게 수집
    #-------------------------------------------------------
    r = session.get(view)
    if r.status_code != 200:
    # 에러코드와 에러메시지 출력
        msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
        print(msg)
        continue

    r.encoding = "utf-8"
    detailSoup = BeautifulSoup(r.text)
    #print(detailSoup)
    # 같은 구조를 갖는 전화번호, 팩스번호, 위치, 이메일주소를 가져온다.
    infoEl = detailSoup.select(".college_info_data dl")
    #print(infoEl)
    #print("-" * 50)
    # 딕셔너리 안에 for문을 돌리는 이유는 
    # 딕셔너리 값 안에 없는 것들을 찾아내서 딕셔너리에 추가하려고
    for info in infoEl:
    #print(info)
        dt = info.select("dt")[0].text.replace(":", "").strip() 
        dd = info.select("dd")[0].text.replace(":", "").strip()
        # info정보안에 dt항 아래에 0번째 택스트를 추출하는데
        # :항목에 앞 뒤 공간이 얼마나 있는 지 
        # 다른 항목들도 같은 공간을 차지하는 지 모르기때문에
        # 그냥 :삭제후 strip으로 같은 조건으로 만든다 
    #print(dt, dd)
    #print("-" * 50)
        college_dict[dt] = dd
    # 딕셔너리에 None값이면 값 추가 하니까 없는 항목에 대해서는 값추가
    #------------------------------------------------------# 
    # 상세보기 URL을 새롭게 수집 (끝)
    #-------------------------------------------------------
    학과목록_리스트.append(college_dict)

df = DataFrame(학과목록_리스트)
df.to_excel("result.xlsx")
df


,학과이름,상세페이지,홈페이지,전화,팩스,위치,이메일
0,한국어문학부,http://www.sookmyung.ac.kr/sookmyungkr/1012/su...,https://korean.sookmyung.ac.kr/,02-710-9310,02-2077-7310,순헌관411,km9310@sookmyung.ac.kr
1,역사문화학과,http://www.sookmyung.ac.kr/sookmyungkr/1013/su...,https://history.sookmyung.ac.kr/,"02-710-9361,9367",02-2077-7311,순헌관412,histo-culture@sookmyung.ac.kr
2,프랑스언어ㆍ문화학과,http://www.sookmyung.ac.kr/sookmyungkr/1014/su...,http://french.sookmyung.ac.kr/,"02-710-9329,9292",02-2077-7314,순헌관314,french@sookmyung.ac.kr
3,중어중문학부,http://www.sookmyung.ac.kr/sookmyungkr/1015/su...,https://chinese.sookmyung.ac.kr/,02-710-9335,02-2077-7315,순헌관311,china9335@sookmyung.ac.kr
4,독일언어ㆍ문화학과,http://www.sookmyung.ac.kr/sookmyungkr/1016/su...,https://german.sookmyung.ac.kr/,02-710-9342,02-2077-7316,순헌관316B,dokmun@sookmyung.ac.kr
5,일본학과,http://www.sookmyung.ac.kr/sookmyungkr/1017/su...,https://japan.sookmyung.ac.kr/,02-710-9885,02-2077-7313,순헌관602,smjapan@sookmyung.ac.kr
6,문헌정보학과,http://www.sookmyung.ac.kr/sookmyungkr/1018/su...,https://lis.sookmyung.ac.kr/,02-710-9373,02-710-9276,순헌관220,an9377@sookmyung.ac.kr
7,문화관광외식학부문화관광학전공,http://www.sookmyung.ac.kr/sookmyungkr/1020/su...,https://cultureandtourism.sookmyung.ac.kr/,02-710-9969,02-2077-7319,새힘관205,ctour@sookmyung.ac.kr
8,문화관광외식학부르꼬르동블루외식경영전공,http://www.sookmyung.ac.kr/sookmyungkr/1021/su...,https://lcb.sookmyung.ac.kr/,02-2077-7335,02-2077-7319,새힘관205,lcb@sookmyung.ac.kr
9,교육학부,http://www.sookmyung.ac.kr/sookmyungkr/1022/su...,https://edu.sookmyung.ac.kr/,02-710-9355,02-2077-7317,순헌관921,edu@sookmyung.ac.kr
